In [1]:
import os
from athina.evals import RagasAnswerCorrectness, RagasAnswerRelevancy, RagasFaithfulness, RagasConciseness
from athina.loaders import Loader
from athina.keys import AthinaApiKey, OpenAiApiKey
from athina.runner.run import EvalRunner
from athina.datasets import yc_query_mini
import pandas as pd

OpenAiApiKey.set_key(os.getenv('OPENAI_API_KEY'))
AthinaApiKey.set_key(os.getenv('ATHINA_API_KEY'))

/Users/akshat_g/athina/repos/athina-evals/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/akshat_g/athina/repos/athina-evals/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_data_ragas = [
    {
        "query": "What is the capital of India?",
        "context": ["India is a south asian country", "Mumbai is the financial capital of India", "New Delhi is the capital of India"],
        "response": "New Delhi is the capital",
    }
]
ragas_dataset = Loader().load_dict(raw_data_ragas)
pd.DataFrame(ragas_dataset)

,query,context,response,expected_response
0,What is the capital of India?,"[India is a south asian country, Mumbai is the...",New Delhi is the capital,None


In [3]:
eval_model = "gpt-3.5-turbo"
RagasConciseness(model=eval_model, api_key=os.getenv('OPENAI_API_KEY'), provider='openai').run_batch(data=ragas_dataset).to_df()


ERROR: Error occurred during eval: There is no current event loop in thread 'ThreadPoolExecutor-0_0'.

ERROR: Error running batch async {'query': 'What is the capital of India?', 'context': ['India is a south asian country', 'Mumbai is the financial capital of India', 'New Delhi is the capital of India'], 'response': 'New Delhi is the capital', 'expected_response': None}: There is no current event loop in thread 'ThreadPoolExecutor-0_0'.



Error logging eval results to Athina: 'NoneType' object has no attribute 'get'
You can view your dataset at: https://app.athina.ai/develop/df1fe6f9-0071-4c21-807b-51410ef145e2


Traceback (most recent call last):
  File "/Users/akshat_g/athina/repos/athina-evals/athina/evals/base_evaluator.py", line 163, in _run_batch_generator_async
    results[index] = future.result()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/concurrent/futures/_base.py", line 438, in result
    return self.__get_result()
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/concurrent/futures/_base.py", line 390, in __get_result
    raise self._exception
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/concurrent/futures/thread.py", line 52, in run
    result = self.fn(*self.args, **self.kwargs)
  File "/Users/akshat_g/athina/repos/athina-evals/athina/evals/ragas/ragas_evaluator.py", line 102, in _evaluate
    raise e
  File "/Users/akshat_g/athina/repos/athina-evals/athina/evals/ragas/ragas_evaluator.py", line 92,

,display_name,failed,grade_reason,runtime,model
0,None,None,None,None,None


In [4]:
# Create batch dataset from list of dict objects
raw_data = yc_query_mini.data

dataset = Loader().load_dict(raw_data)
pd.DataFrame(dataset)

,query,context,response,expected_response
0,What are some successful companies that went through YC?,"[Y Combinator has invested in companies in various fields like FinTech, Healthcare, AI, etc.]","Airbnb, Dropbox, Stripe, Reddit, Coinbase, Instacart.",Airbnb and Stripe are 2 of the successful companies that went through YC.
1,In which city is YC located?,"[Y Combinator is located in Mountain View, California.]",Y Combinator is located in San Francisco,"YC is located in Mountain View, California."
2,How much equity does YC take?,[Y Combinator invests $500k in 200 startups twice a year.],YC invests $150k for 7%.,I cannot answer this question as I do not have enough information.
3,How much equity does YC take?,[Y Combinator invests $500k in 200 startups twice a year.],I cannot answer this question as I do not have enough information.,I cannot answer this question as I do not have enough information.
4,Who founded YC and when was it founded?,"[Y Combinator was founded in March 2005 by Paul Graham, Jessica Livingston, Trevor Blackwell, and Robert Tappan Morris.]",Y Combinator was founded in 2005,"Y Combinator was founded in March 2005 by Paul Graham, Jessica Livingston, Trevor Blackwell, and Robert Tappan Morris."
5,Does Y Combinator invest in startups outside the US?,[Y Combinator invests in startups from all over the world.],"Yes, Y Combinator invests in international startups as well as US startups.","Yes, Y Combinator invests in startups from all over the world."
6,How much does YC invest in startups?,[YC invests $150k for 7%.],$150k,YC invests $150k for 7%.
7,What is YC's motto?,[Y Combinator's motto is 'Make something people want'.],Make something people want,Make something people want


In [5]:
# Run the eval suite
eval_model = "gpt-4-1106-preview"
eval_suite = [
    RagasAnswerCorrectness(model=eval_model, api_key=os.getenv('OPENAI_API_KEY'), provider='openai'),
    RagasFaithfulness(model=eval_model, api_key=os.getenv('OPENAI_API_KEY'), provider='openai'),
    RagasAnswerRelevancy(model=eval_model, api_key=os.getenv('OPENAI_API_KEY'), provider='openai'),
]

# Run the evaluation suite
batch_eval_result = EvalRunner.run_suite(
    evals=eval_suite,
    data=dataset,
    max_parallel_evals=1,   # If you increase this, you may run into rate limits
)

pd.DataFrame(batch_eval_result)

TypeError: __init__() missing 3 required positional arguments: 'api_key', 'model', and 'provider'